In [1]:
import pandas as pd
import numpy as np

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# verify GPU
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Convert to DataFrame for EDA

In [4]:
import pandas as pd

# Read the data from the text file
with open("/content/drive/MyDrive/266_final/data/dataset1_train.txt", "r") as file:
    lines = file.readlines()

# Define an empty list to store the data
data = []

# Iterate over each line in the file
for line in lines:
    # Split the line by spaces
    parts = line.strip().split()

    # Check if the line has the expected number of elements
    if len(parts) == 9:
        # Extract the values from the line
        text_file_name = parts[0]
        sentence_line_number = int(parts[1])
        sentence_word_index = int(parts[2])
        sentence_seq = parts[3]
        start_token = int(parts[4])
        end_token = int(parts[5])
        original_word = parts[6]
        word = parts[7]
        label = parts[8]

        # Append the values as a tuple to the data list
        data.append((text_file_name, sentence_line_number, sentence_word_index, sentence_seq,
                     start_token, end_token, original_word, word, label))

# Create a DataFrame from the data list with appropriate column names
df = pd.DataFrame(data, columns=['text_file_name', 'sentence_line_number', 'sentence_word_index',
                                 'sentence_seq', 'start_token', 'end_token', 'original_word',
                                 'word', 'label'])




In [5]:
df.head()

,text_file_name,sentence_line_number,sentence_word_index,sentence_seq,start_token,end_token,original_word,word,label
0,data/training_20180910/110727.txt,1,0,NA,0,9,Admission,Admission,O
1,data/training_20180910/110727.txt,1,1,NA,10,14,Date,Date,O
2,data/training_20180910/110727.txt,1,2,NA,14,15,:,:,O
3,data/training_20180910/110727.txt,1,3,NA,17,18,[,[,O
4,data/training_20180910/110727.txt,1,4,NA,18,19,*,*,O


In [6]:
len(df)

895141

In [7]:
df['sentence_line_number'].nunique()

1053

In [8]:
df['label'].value_counts()

O              802045
B-Drug          16222
I-Frequency     13023
I-Dosage         8779
B-Strength       6691
B-Form           6647
I-Strength       6617
B-Frequency      6279
B-Route          5475
I-Drug           4298
B-Dosage         4221
I-Form           4173
B-Reason         3791
I-Reason         3125
I-Duration       1034
B-ADE             956
I-ADE             776
B-Duration        592
I-Route           397
Name: label, dtype: int64

In [9]:
df[df['sentence_line_number'] == 1]

,text_file_name,sentence_line_number,sentence_word_index,sentence_seq,start_token,end_token,original_word,word,label
0,data/training_20180910/110727.txt,1,0,NA,0,9,Admission,Admission,O
1,data/training_20180910/110727.txt,1,1,NA,10,14,Date,Date,O
2,data/training_20180910/110727.txt,1,2,NA,14,15,:,:,O
3,data/training_20180910/110727.txt,1,3,NA,17,18,[,[,O
4,data/training_20180910/110727.txt,1,4,NA,18,19,*,*,O
...,...,...,...,...,...,...,...,...,...
893314,data/training_20180910/100883.txt,1,23,NA,64,65,-,-,O
893315,data/training_20180910/100883.txt,1,24,NA,65,66,3,ORDINAL,O
893316,data/training_20180910/100883.txt,1,25,NA,66,67,*,*,O
893317,data/training_20180910/100883.txt,1,26,NA,67,68,*,*,O


### Label labels

In [10]:
# Split labels based on whitespace and turn them into a list
labels = [i.split() for i in df['label'].values.tolist()]

# Check how many labels are there in the dataset
unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]

print(unique_labels)

{'O', 'B-Route', 'B-Duration', 'B-Form', 'B-ADE', 'I-Route', 'I-Duration', 'I-ADE', 'B-Drug', 'I-Frequency', 'B-Reason', 'I-Dosage', 'B-Dosage', 'B-Frequency', 'I-Strength', 'I-Drug', 'I-Reason', 'I-Form', 'B-Strength'}


In [11]:
# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

{'B-ADE': 0, 'B-Dosage': 1, 'B-Drug': 2, 'B-Duration': 3, 'B-Form': 4, 'B-Frequency': 5, 'B-Reason': 6, 'B-Route': 7, 'B-Strength': 8, 'I-ADE': 9, 'I-Dosage': 10, 'I-Drug': 11, 'I-Duration': 12, 'I-Form': 13, 'I-Frequency': 14, 'I-Reason': 15, 'I-Route': 16, 'I-Strength': 17, 'O': 18}


## Experimenting with formats for BERT

In [12]:
train_texts = []
train_labels = []

# Read the text file line by line
with open('/content/drive/MyDrive/266_final/data/dataset1_train.txt', 'r', encoding='utf-8') as file:
    current_text = []  # To store tokens of the current text
    current_labels = []  # To store labels of the current text
    for line in file:
        if line.strip() == '':  # Empty line signifies end of text
            train_texts.append(current_text)
            train_labels.append(current_labels)
            current_text = []
            current_labels = []
        else:
            parts = line.strip().split()
            token = parts[-3]  # Token is second-to-last part
            label = parts[-1]   # Label is last part
            current_text.append(token)
            current_labels.append(label)

# Check the first few samples
print(train_texts[:5])
print(train_labels[:5])


[['Admission', 'Date', ':', '[', '*', '*', '2202', '-', '1', '-', '8', '*', '*', ']', 'Discharge', 'Date', ':', '[', '*', '*', '2202', '-', '2', '-', '1', '*', '*', ']'], ['Date', 'of', 'Birth', ':', '[', '*', '*', '2163', '-', '9', '-', '18', '*', '*', ']', 'Sex', ':', 'M'], ['Service', ':', 'MEDICINE'], ['Allergies', ':', 'Keflex', '/', 'Orencia', '/', 'Remicade'], ['Attending', ':', '[', '*', '*', 'First', 'Name3', '(', 'LF', ')', '2751', '*', '*', ']', 'Chief', 'Complaint', ':', 'L', 'leg', 'pain', 'and', 'erythema']]
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O'], ['O', 'O', 'B-Drug', 'O', 'B-Drug', 'O', 'B-Drug'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]


In [13]:
train_texts[4]

['Attending',
 ':',
 '[',
 '*',
 '*',
 'First',
 'Name3',
 '(',
 'LF',
 ')',
 '2751',
 '*',
 '*',
 ']',
 'Chief',
 'Complaint',
 ':',
 'L',
 'leg',
 'pain',
 'and',
 'erythema']

In [14]:
train_labels[4]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

### Not sure if this is the correct format. Let's look into another way to turn it into a sentence.

In [15]:
df['sentence'] = df[['text_file_name', 'sentence_line_number', 'original_word', 'label']].groupby(['text_file_name', 'sentence_line_number'])['original_word'].transform(lambda x: ' '.join(x))

In [16]:
df['word_labels'] = df[['text_file_name', 'sentence_line_number', 'original_word', 'label']].groupby(['text_file_name', 'sentence_line_number'])['label'].transform(lambda x: ','.join(x))


In [17]:
sentence_level_data = df[["sentence", "word_labels"]].drop_duplicates().reset_index(drop=True)
sentence_level_data.head()

,sentence,word_labels
0,Admission Date : [ * * 2202 - 1 - 8 * * ] Disc...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,Date of Birth : [ * * 2163 - 9 - 18 * * ] Sex : M,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O"
2,Service : MEDICINE,"O,O,O"
3,Allergies :,"O,O"
4,Keflex / Orencia / Remicade,"B-Drug,O,B-Drug,O,B-Drug"


In [18]:
sentence_level_data.iloc[50].sentence

'otherwise negative .'

In [19]:
sentence_level_data.iloc[50].word_labels

'O,O,O'

In [20]:
data = sentence_level_data.copy()

### It looks reasonable, let's proceed with training

In [21]:
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
import os




In [22]:
# from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
import os
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "emilyalsentzer/Bio_ClinicalBERT"  # Bio_ClinicalBERT model identifier
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(labels_to_ids))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at emilyalsentzer/Bio_ClinicalBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')


In [24]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels
        sentence = self.data.sentence[index].strip().split()
        word_labels = self.data.word_labels[index].split(",")

        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        # BertTokenizerFast provides a handy "return_offsets_mapping" functionality for individual tokens
        encoding = self.tokenizer(sentence,
                             return_offsets_mapping=True,
                             padding='max_length',
                             is_split_into_words=True,
                             truncation=True,
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels]
        # code based on https://huggingface.co/transformers/custom_datasets.html#tok-ner
        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100

        # set only labels whose first offset position is 0 and the second is not 0
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)

        return item

  def __len__(self):
        return self.len

In [25]:
train_size = 0.8
train_dataset = data.sample(frac=train_size,random_state=200)
test_dataset = data.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(data.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (66944, 2)
TRAIN Dataset: (53555, 2)
TEST Dataset: (13389, 2)


In [26]:
training_set[0]

{'input_ids': tensor([  101,  3879,   119,  1224,  1142,  1285,  1119,  1108,  1195,  6354,
          1181,  1121, 14516, 13759,   117, 21167,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [27]:
for token, label in zip(tokenizer.convert_ids_to_tokens(training_set[0]["input_ids"]), training_set[0]["labels"]):
  print('{0:10}  {1}'.format(token, label))

[CLS]       -100
condition   18
.           18
later       18
this        18
day         18
he          18
was         18
we          18
##ane       -100
##d         -100
from        18
se          18
##dation    -100
,           18
awoke       18
[SEP]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD]       -100
[PAD

In [28]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [29]:
# model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, el

In [30]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(3.1163, device='cuda:0', grad_fn=<NllLossBackward0>)

In [31]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 128, 19])

In [32]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [33]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)


        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
        tr_loss += loss.item()

        # outputs = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)
        # loss = outputs["loss"]
        # tr_logits = outputs["logits"]

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))

        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [34]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.958996057510376
Training loss per 100 training steps: 1.0380283643187274
Training loss per 100 training steps: 0.7250493088116249
Training loss per 100 training steps: 0.5803420235622563
Training loss per 100 training steps: 0.482567963028963
Training loss per 100 training steps: 0.4212975237033369
Training loss per 100 training steps: 0.3850111600085969
Training loss per 100 training steps: 0.3537028963108296
Training loss per 100 training steps: 0.3268136583843016
Training loss per 100 training steps: 0.30858879979156145
Training loss per 100 training steps: 0.29061335963629686
Training loss per 100 training steps: 0.2781920375604416
Training loss per 100 training steps: 0.2687524695763229
Training loss per 100 training steps: 0.25812661126406317
Training loss per 100 training steps: 0.2486846257894593
Training loss per 100 training steps: 0.24051721798188408
Training loss per 100 training steps: 0.23457436419882086
Training l

In [35]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)

            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels, return_dict=False)

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)

            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)

            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [36]:
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.08211545646190643
Validation loss per 100 evaluation steps: 0.05383089311233039
Validation loss per 100 evaluation steps: 0.06638102648950318
Validation loss per 100 evaluation steps: 0.07760291869819405
Validation loss per 100 evaluation steps: 0.0747814670469591
Validation loss per 100 evaluation steps: 0.07466279936250543
Validation loss per 100 evaluation steps: 0.0720229491102548
Validation loss per 100 evaluation steps: 0.07395508084305863
Validation loss per 100 evaluation steps: 0.07193509797080866
Validation loss per 100 evaluation steps: 0.07114755149864369
Validation loss per 100 evaluation steps: 0.07157514905871916
Validation loss per 100 evaluation steps: 0.07152232416784078
Validation loss per 100 evaluation steps: 0.07293472856403481
Validation loss per 100 evaluation steps: 0.07275725735722702
Validation loss per 100 evaluation steps: 0.07312377783594746
Validation loss per 100 evaluation steps: 0.07212534619855997
Validation

In [37]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 920.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=ac82962a891ce842e024c27d2afc0cad29abc63f885610a5773f4561553ad5fd
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [38]:
from seqeval.metrics import classification_report

print(classification_report([labels], [predictions]))

              precision    recall  f1-score   support

         ADE       0.44      0.18      0.26       216
      Dosage       0.84      0.92      0.88       832
        Drug       0.92      0.93      0.93      3163
    Duration       0.75      0.73      0.74       134
        Form       0.94      0.89      0.91      1252
   Frequency       0.86      0.90      0.88      1203
      Reason       0.55      0.46      0.50       738
       Route       0.93      0.95      0.94       992
    Strength       0.94      0.95      0.95      1265

   micro avg       0.88      0.87      0.88      9795
   macro avg       0.80      0.77      0.78      9795
weighted avg       0.87      0.87      0.87      9795



In [40]:
import os

directory = "/content/drive/MyDrive/266_final/model_clinicalbert"

if not os.path.exists(directory):
    os.makedirs(directory)

# save vocabulary of the tokenizer
tokenizer.save_vocabulary(directory)
# save the model weights and its configuration file
model.save_pretrained(directory)
print('All files saved')
print('This tutorial is completed')

All files saved
This tutorial is completed


Resources

### https://github.com/lcampillos/Medical-NER/blob/master/bert_ner.ipynb
### https://medium.com/analytics-vidhya/bio-tagged-text-to-original-text-99b05da6664